In [5]:
import csv
import os
import requests
import json
import time
import io
from datetime import datetime
from collections import OrderedDict
os.chdir(r"D:\Users\snangare\Sandesh\API")
API_URI='https://amgen-etmf.veevavault.com/api/'
API_VERSION = 'v19.1'
BATCH_SIZE=500
FILE_NAME='DocumentIDs.csv'
SUCCESS_CSV = []
FAILURE_CSV = []
CLIENT_ID='Export_ObjectRoles'
LOG_DIRECTORY='logs'
# Authenticate with Vault and get the session id
# https://developer.veevavault.com/api/19.1/#authentication
def authenticate():
	action = '/auth'
	endpoint = API_URI + API_VERSION + action
	parameters = {
		'username':'snangare@amgensbx.com',
		'password':'Test@54321'
	}
	try: 
		response = requests.post(endpoint, data=parameters).json()
		if response['responseStatus'] != 'SUCCESS':
			raise Exception()
		return response['sessionId']
	except:
		error_message = response['errorType'] + ', Reason: ' + response['responseMessage']
		print (error_message)
		
def get_document_attachment(session_id, document_id):
	API = '/objects/documents/{document_id}/attachments'.format(
		document_id=document_id, 
		)
	endpoint = API_URI + API_VERSION + API

	headers = {
		'Authorization': session_id,
		'Content-type': 'text/csv'
		}
	try: 
		response = requests.get(endpoint, headers=headers).json()
		result = OrderedDict()
		result.update({
		'id' : document_id,
		})

		if response['responseStatus'] == 'SUCCESS':
			result.update({'status':response['responseStatus']})
			list=response['data']
			result.update({'Total Attachments':len(list)})
			
			for i in range (len(list)):
				Attachmentno='Attachment-' + str(i+1)
				AttachmentID='DocID of Attachment-' + str(i+1)
				AttachmentSize='Size of Attachment-' + str(i+1)
				print(response['data'][i]['size__v'])
				print(response['data'][i]['id'])
				result.update({Attachmentno:response['data'][i]['filename__v']})
				result.update({AttachmentID:response['data'][i]['id']})
				result.update({AttachmentSize:response['data'][i]['size__v']})
				#result.update({'Attachment-2':response['data'][1]['filename__v']})
			
			SUCCESS_CSV.append(result)   
					
		else:
			result.update({'status':response['errors']})
			FAILURE_CSV.append(result)

	except Exception as e:
		print ('Exception:' + str(e))
		#SUCCESS_CSV.append(result)  
		#FAILURE_CSV.append(result)
		
def writeDictToCSV(csv_file,csv_columns,dict_data):
	with open(csv_file, 'w', encoding='utf-8') as csvfile:
		writer = csv.DictWriter(csvfile, fieldnames=csv_columns, delimiter=',', lineterminator = '\n',quoting=csv.QUOTE_ALL)
		writer.writeheader()
		try:
			for data in dict_data:
				#print(data)
				writer.writerow(data) 
			
		except Exception as e:
			print ('Exception:' + str(e))
			 
# Write the success and failure into a log
def write_csv_log(csv_columns):
	#print('COLUMN: ' + str(csv_columns))
	current_time = datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")
	filename=LOG_DIRECTORY + '/Documents_AttachmentReport' + '_SUCCESS_' + str(current_time) + '.csv'
	if len(SUCCESS_CSV) > 0:
		writeDictToCSV(filename, csv_columns, SUCCESS_CSV)
		
	filename=LOG_DIRECTORY + '/Documents_AttachmentReport' + '_FAILURE_' + str(current_time) + '.csv'
	if len(FAILURE_CSV) > 0:
		writeDictToCSV(filename, csv_columns, FAILURE_CSV)
		
			
def execute():
	session_id = authenticate()
	column_names = None
	if session_id is not None:
		with open(FILE_NAME) as csvfile:
			reader = csv.reader(csvfile, delimiter=',')
			row_number = 0;
			for row in reader:
				print('Reading Row Number: ' + str(row_number))
				if (row_number==0):
					column_names = row
					column_names.append('status')
					column_names.append('Total Attachments')
					
					for i in range (25):
						Attachmentno='Attachment-' + str(i+1)
						AttachmentID='DocID of Attachment-' + str(i+1)
						AttachmentSize ='Size of Attachment-' + str(i+1)
						
						column_names.append(Attachmentno)
						column_names.append(AttachmentID)
						column_names.append(AttachmentSize)

				else:
					document_id = None
					data = {}
					for i in range(0, len(row)):
						column_name = column_names[i]
						cell = row[i]
						if column_name == 'id':
							document_id = cell
							#print(object_id)		
					if document_id is not None:
						get_document_attachment(session_id, document_id)
						if len(SUCCESS_CSV) == 50000:
							write_csv_log(column_names)
							SUCCESS_CSV.clear()
						if len(FAILURE_CSV) == 50000:
							write_csv_log(column_names)
							FAILURE_CSV.clear()
					else:
						print ('[ERROR]: Object ID is not Vaild')
				row_number += 1	
			write_csv_log(column_names)

# Main Execution		
print('.........................Extracting Documents Attachment Information..............................')
execute()
print('.........................Attachment Export Job is Completed.......................................')

.........................Extracting Documents Attachment Information..............................
Reading Row Number: 0
Reading Row Number: 1
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 2
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 3
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 4
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 5
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 6
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 7
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 8
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 9
Exception:Extra data: line 1 column 147 (char 146)
Reading Row Number: 10
Exception:Extra data: line 1 column 147 (char 146)
.........................Attachment Export Job is Completed.......................................
